<a href="https://colab.research.google.com/github/alorenz465446/NYC_Parking_Tickets/blob/Beatriz_Marte/NYC_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
import pandas as pd

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving NYC_Tickets_2017.csv to NYC_Tickets_2017 (1).csv


In [ ]:
nyc_df = pd.read_csv("NYC_Tickets_2017.csv", low_memory=False)
nyc_df 

,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Violation Time,Violation County,Street Name,Law Section,Sub Division,Vehicle Color,Vehicle Year,Violation Description
0,PAS,7/10/2016,7,SUBN,TOYOT,V,NaN,0,0,0,NaN,0143A,BX,ALLERTON AVE (W/B) @,1111,D,GY,2001,FAILURE TO STOP AT RED LIGHT
1,PAS,7/8/2016,7,SUBN,TOYOT,V,NaN,0,0,0,NaN,0400P,BX,ALLERTON AVE (W/B) @,1111,D,GY,2001,FAILURE TO STOP AT RED LIGHT
2,PAS,8/23/2016,5,SUBN,FORD,V,NaN,0,0,0,NaN,0233P,BX,SB WEBSTER AVE @ E 1,1111,C,BK,2004,BUS LANE VIOLATION
3,COM,6/14/2017,47,REFG,MITSU,T,14.0,14,14,359594,T102,1120A,NY,7th Ave,408,l2,WH,2007,47-Double PKG-Midtown
4,COM,11/21/2016,69,DELV,INTER,T,13.0,13,13,364832,T102,0555P,NY,6th Ave,408,h1,WHITE,2007,69-Failure to Disp Muni Recpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,PAS,6/9/2017,36,4DSD,CHEVR,V,NaN,0,0,0,NaN,0200P,QN,WB HILLSIDE AVE @ 23,1180,B,RD,2003,PHTO SCHOOL ZN SPEED VIOLATION
1048571,PAS,12/11/2016,71,4DSD,NISSA,T,81.0,81,81,357358,T301,1001A,K,Putnam Ave,408,j6,GY,2007,71A-Insp Sticker Expired (NYS)
1048572,PAS,4/13/2017,38,4DSD,HONDA,T,60.0,60,60,362530,T302,0202P,K,Coney Island Ave,408,h1,BL,2015,38-Failure to Display Muni Rec
1048573,PAS,4/4/2017,21,SUBN,VOLVO,T,83.0,83,83,364015,T301,0907A,K,Putnam Ave,408,d1,GY,2004,21-No Parking (street clean)


# Encode Categorical Data 
Use one One_Hot_Encoder 

In [ ]:
# Generate our categorical variable list
df_categorical = nyc_df.dtypes[nyc_df.dtypes == "object"].index.tolist()


In [ ]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_categorical]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(df_categorical)
encode_df.head()


Consider Binning. 

Sample code below: 

```

# Look at INCOME_AMT value counts for binning
INCOME_AMT_count=application_df['INCOME_AMT'].value_counts()

# Visualize the value counts of INCOME_AMT
INCOME_AMT_count.plot.density()

---

# Determine which values to replace if counts are less than ...?
replace_INCOME_AMT = list(INCOME_AMT_count[INCOME_AMT_count < 500].index)


# Replace in dataframe
for app in replace_INCOME_AMT:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(app,"5M+")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()
```

# Start with a Random Forest model to check for correlations. 

Note:
(used homework 17.8.2-3, named random_forest_model in JN)
- BM

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


### Preprocess the Data


In [ ]:
# Define the features set.
X = nyc_df.copy()
X = X.drop(['Violation Code', 'Violation Description'], axis=1)
X.head()


In [ ]:
# Define the target set.
y = df_loans["Violation Code"].ravel()
y[:5]


In [ ]:
# Split data into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Fit the Random Forest Model


In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128) 


> The `n_estimators` will allow us to set the number of trees that will be created by the algorithm. Generally, the higher number makes the predictions stronger and more stable, but can slow down the output because of the higher training time allocated. The best practice is to use between 64 and 128 random forests, though higher numbers are quite common despite the higher training time. 

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


### Make Predictions Using the Testing Data


In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

### Evaluate the Model


In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df


In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)


In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("-")
print("Classification Report")
print(classification_report(y_test, predictions))


### Rank the Importance of Features


In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


In [ ]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
